In [1]:
import pandas as pd
import geopandas as gpd
import json
from tqdm.auto import tqdm

In [2]:
import os
os.chdir('Science-Impacts')
os.system('git pull')


From github.com:mjharris95/Science-Impacts
   38157d2..4777e48  main       -> origin/main


Updating 38157d2..4777e48
Fast-forward
 .gitignore                          |      3 +
 NIH_test.Rmd                        |     94 +-
 data/NIH_impact_cong_district_1.csv |    472 -
 data/NIH_impact_county_1.csv        |   3136 -
 data/NIH_impact_state_1.csv         |     53 -
 data/OD_countySum001_2016.csv       |  14587 +-
 data/county_pop.csv                 |      9 +-
 data/nih_terminations_airtable.csv  |   3625 +-
 new_names.csv                       |     96 -
 output/NIH_clean_fips.csv           | 131322 +++---
 output/NIH_impact_cong.csv          |    874 +-
 output/NIH_impact_county.csv        |   6283 +-
 output/NIH_impact_state.csv         |    102 +-
 output/county_commute_NIH.csv       | 790110 +++++++++++++++++++++++++++++++++
 output/static_loss_county.csv       |    432 +
 output/terminated_points.csv        |    602 +-
 possible-repeats                    |     43 -
 term_data/repeated_orgs.csv         |    380 +-
 18 files changed, 870331 insertions(+), 81892 dele

0

# File Loading

In [ ]:
# (base) colin@robco:~/code/scimap-data/Science-Impacts/output$ ls
# NIH_impact_cong.csv  NIH_impact_county.csv  NIH_impact_state.csv  terminated_points.csv

In [3]:

# base_dir = "/home/colin/code/scimap-data/Science-Impacts"
# county_idc_file = os.path.join(base_dir, "NIH_impact_IDC_county.csv")
# state_idc_file = os.path.join(base_dir, "state_impacts_IDC.csv")
# congressional_idc_file = os.path.join(base_dir, "NIH_impact_IDC_cong_district.csv")

# base_term_dir = "/home/colin/code/scimap-data/Science-Impacts/term_data"
# county_term_file = os.path.join(base_term_dir, "term_impact_county.csv")
# state_term_file = os.path.join(base_term_dir, "state_term_impacts.csv")
# congressional_term_file = os.path.join(base_term_dir, "term_impact_cong_district.csv")
base_folder = "/home/colin/code/scimap-data/Science-Impacts/output"
county_total_file = os.path.join(base_folder, "NIH_impact_county.csv")
state_total_file = os.path.join(base_folder, "NIH_impact_state.csv")
congressional_total_file = os.path.join(base_folder, "NIH_impact_cong.csv")
# county_total_file = "/home/colin/code/scimap-data/Science-Impacts/output/NIH_TOTAL_COUNTY.csv"
# state_total_file = "/home/colin/Downloads/NIH_TOTAL_STATE.csv"
# congressional_total_file = "/home/colin/Downloads/NIH_TOTAL_CONG.csv"

# Try different encodings
# df_idc_county = pd.read_csv(county_idc_file, encoding='latin-1')
# df_idc_state = pd.read_csv(state_idc_file, encoding='latin-1')
# df_idc_cong = pd.read_csv(congressional_idc_file, encoding='latin-1')

# df_term_county = pd.read_csv(county_term_file, encoding='latin-1')
# df_term_state = pd.read_csv(state_term_file, encoding='latin-1')
# df_term_cong = pd.read_csv(congressional_term_file, encoding='latin-1')


df_total_county = pd.read_csv(county_total_file, encoding='latin-1')
df_total_state = pd.read_csv(state_total_file, encoding='latin-1')
df_total_cong = pd.read_csv(congressional_total_file, encoding='latin-1')


## Process County

In [4]:
counties = gpd.read_file('/home/colin/code/scimap-data/inputs/merged_data_counties_CLIP_Compress.geojson') # Example, replace with your shapefile path
counties = counties.to_crs(epsg=4326) # Project to WGS84 (lat/lon)
# Merge the dataframes based on the FIPS code
# merged_df_county_idc = pd.merge(df_idc_county, counties[['FIPS', 'geometry']], on='FIPS', how='left')
# merged_df_county_term = pd.merge(df_term_county, counties[['FIPS', 'geometry']], on='FIPS', how='left')
merged_df_county_total = pd.merge(df_total_county, counties[['FIPS', 'geometry']], on='FIPS', how='left')

In [5]:

merged_df_county_total

,state,state_code,FIPS,county,pop_2024,terminated_loss,terminated_loss_noself,terminated_econ_loss,terminated_job_loss,terminated_econ_loss_noself,...,IDC_loss_log,IDC_econ_loss_log,IDC_job_loss_log,grant_funds_log,grant_funds_econ_log,combined_loss_log,combined_econ_loss_log,combined_job_loss_log,combined_loss_noself_log,geometry
0,Alabama,AL,1001,Autauga County,61464,75226.263114,75159.187529,1.925792e+05,0.831778,1.924075e+05,...,12.251303,13.191307,1.198069,14.220517,15.160523,12.546923,13.486693,1.413133,12.546687,"POLYGON ((-86.41312 32.70739, -86.43963 32.708..."
1,Alabama,AL,1003,Baldwin County,261608,104062.458865,103855.263129,2.663999e+05,1.150621,2.658695e+05,...,13.067809,14.007815,1.830175,14.907743,15.847750,13.258913,14.198559,1.992671,13.258553,"POLYGON ((-87.76515 31.29735, -87.76522 31.296..."
2,Alabama,AL,1005,Barbour County,24358,26322.573987,26295.936798,6.738579e+04,0.291049,6.731760e+04,...,11.003850,11.943847,0.509568,12.885751,13.825757,11.353403,12.293090,0.663904,11.353090,"POLYGON ((-85.05603 32.06306, -85.07113 32.063..."
3,Alabama,AL,1007,Bibb County,22258,72105.593602,72087.614874,1.845903e+05,0.797273,1.845443e+05,...,12.271008,13.211012,1.211868,14.271185,15.211192,12.551670,13.491612,1.416857,12.551607,"POLYGON ((-87.06574 33.24691, -87.06812 33.246..."
4,Alabama,AL,1009,Blount County,60163,479424.325520,479379.304577,1.227326e+06,5.301005,1.227211e+06,...,14.024659,14.964665,2.683018,16.026435,16.966442,14.341700,15.281680,2.981287,14.341673,"POLYGON ((-86.45302 34.25932, -86.48207 34.260..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,Wyoming,WY,56037,Sweetwater County,41273,3880.586432,3809.895314,9.934301e+03,0.042908,9.753332e+03,...,9.884377,10.824353,0.196342,11.794970,12.734972,10.058952,10.995916,0.229163,10.055935,"POLYGON ((-109.04386 42.26368, -109.05348 42.2..."
3140,Wyoming,WY,56039,Teton County,23272,1530.969844,1529.834734,3.919283e+03,0.016928,3.916377e+03,...,9.244296,10.184244,0.108296,11.131391,12.071390,9.378758,10.318636,0.122953,9.378681,"POLYGON ((-110.05318 44.00802, -110.05322 44.0..."
3141,Wyoming,WY,56041,Uinta County,20621,1268.792710,1231.875382,3.248109e+03,0.014029,3.153601e+03,...,9.356855,10.296810,0.120452,11.224981,12.164980,9.456686,10.393756,0.131938,9.453797,"POLYGON ((-110.048 41.57802, -110.06279 41.577..."
3142,Wyoming,WY,56043,Washakie County,7662,208.594366,207.433776,5.340016e+02,0.002306,5.310305e+02,...,8.476676,9.416556,0.051718,10.429108,11.369098,8.517963,9.457650,0.053830,8.517765,"POLYGON ((-107.14681 44.16567, -107.16386 44.1..."


In [6]:
# features = []

# for index, row in tqdm(merged_df_county_idc.iterrows(), desc="Processing into GeoJSON (IDC COUNTY)", total=len(merged_df_county_idc)):
#     # Convert the Shapely geometry to GeoJSON format
#     geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
#     geometry = geometry['features'][0]['geometry']

#     # Create a feature dictionary
#     feature = {
#         "type": "Feature",
#         "geometry": geometry,
#         "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
#     }
#     features.append(feature)

# # Create the GeoJSON feature collection
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Save the GeoJSON to a file
# with open("/home/colin/code/scimap-data/outputs/merged_data_counties_idc.geojson", "w") as f:
#     json.dump(geojson, f)


# features = []

# for index, row in tqdm(merged_df_county_term.iterrows(), desc="Processing into GeoJSON (TERM COUNTY)", total=len(merged_df_county_term)):
#     # Convert the Shapely geometry to GeoJSON format
#     geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
#     geometry = geometry['features'][0]['geometry']

#     # Create a feature dictionary
#     feature = {
#         "type": "Feature",
#         "geometry": geometry,
#         "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
#     }
#     features.append(feature)

# # Create the GeoJSON feature collection
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Save the GeoJSON to a file
# with open("/home/colin/code/scimap-data/outputs/merged_data_counties_term.geojson", "w") as f:
#     json.dump(geojson, f)



features = []

for index, row in tqdm(merged_df_county_total.iterrows(), desc="Processing into GeoJSON (TOTAL COUNTY)", total=len(merged_df_county_total)):
    # Convert the Shapely geometry to GeoJSON format
    geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
    geometry = geometry['features'][0]['geometry']


    columns_to_drop = [
        "combined_loss_noself",
        "terminated_loss_log",
        "terminated_loss_noself_log",
        "terminated_econ_loss_log",
        "terminated_job_loss_log",
        "terminated_econ_loss_noself_log",
        "terminated_job_loss_noself_log",
        "IDC_loss_log",
        "IDC_econ_loss_log",
        "IDC_job_loss_log",
        "grant_funds_log",
        "grant_funds_econ_log",
        "combined_job_loss_log"
    ]
    for c in columns_to_drop:
        row = row.drop(c)
    # Create a feature dictionary
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
    }
    features.append(feature)

# Create the GeoJSON feature collection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON to a file
with open("/home/colin/code/scimap-data/outputs/merged_data_counties_total.geojson", "w") as f:
    json.dump(geojson, f)

Processing into GeoJSON (TOTAL COUNTY):   0%|          | 0/3144 [00:00<?, ?it/s]

## Processing States

In [7]:
states = gpd.read_file('/home/colin/code/scimap-data/inputs/merged_data_states_CLIP.geojson') # Example, replace with your shapefile path
states = states.to_crs(epsg=4326) # Project to WGS84 (lat/lon)
# Merge the dataframes based on the FIPS code
# merged_df_state_idc = pd.merge(df_idc_state, states[['state_code', 'geometry']], on='state_code', how='left')
# merged_df_state_term = pd.merge(df_term_state, states[['state_code', 'geometry']], on='state_code', how='left')
merged_df_state_total = pd.merge(df_total_state, states[['state_code', 'geometry']], on='state_code', how='left')

In [8]:
# features = []

# for index, row in tqdm(merged_df_state_idc.iterrows(), desc="Processing into GeoJSON (IDC State)", total=len(merged_df_state_idc)):
#     # Convert the Shapely geometry to GeoJSON format
#     geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
#     geometry = geometry['features'][0]['geometry']

#     # Create a feature dictionary
#     feature = {
#         "type": "Feature",
#         "geometry": geometry,
#         "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
#     }
#     features.append(feature)

# # Create the GeoJSON feature collection
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Save the GeoJSON to a file
# with open("/home/colin/code/scimap-data/outputs/merged_data_states_idc.geojson", "w") as f:
#     json.dump(geojson, f)


# features = []

# for index, row in tqdm(merged_df_state_term.iterrows(), desc="Processing into GeoJSON (TERM State)", total=len(merged_df_state_idc)):
#     # Convert the Shapely geometry to GeoJSON format
#     geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
#     geometry = geometry['features'][0]['geometry']

#     # Create a feature dictionary
#     feature = {
#         "type": "Feature",
#         "geometry": geometry,
#         "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
#     }
#     features.append(feature)

# # Create the GeoJSON feature collection
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Save the GeoJSON to a file
# with open("/home/colin/code/scimap-data/outputs/merged_data_states_term.geojson", "w") as f:
#     json.dump(geojson, f)



features = []

for index, row in tqdm(merged_df_state_total.iterrows(), desc="Processing into GeoJSON (TOTAL State)", total=len(merged_df_state_total)):
    # Convert the Shapely geometry to GeoJSON format
    geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
    geometry = geometry['features'][0]['geometry']

    # Create a feature dictionary
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
    }
    features.append(feature)

# Create the GeoJSON feature collection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON to a file
with open("/home/colin/code/scimap-data/outputs/merged_data_states_total.geojson", "w") as f:
    json.dump(geojson, f)

Processing into GeoJSON (TOTAL State):   0%|          | 0/51 [00:00<?, ?it/s]

In [9]:
df_total_cong.columns

Index(['state', 'state_code', 'GEOID', 'terminated_loss',
       'terminated_loss_noself', 'terminated_econ_loss', 'terminated_job_loss',
       'terminated_econ_loss_noself', 'terminated_job_loss_noself', 'IDC_loss',
       'IDC_econ_loss', 'IDC_job_loss', 'grant_funds', 'grant_funds_econ',
       'overlap_loss', 'overlap_econ_loss', 'overlap_job_loss',
       'overlap_loss_noself', 'overlap_econ_loss_noself',
       'overlap_job_loss_noself', 'combined_loss', 'combined_econ_loss',
       'combined_job_loss', 'combined_loss_noself', 'terminated_loss_log',
       'terminated_loss_noself_log', 'terminated_econ_loss_log',
       'terminated_job_loss_log', 'terminated_econ_loss_noself_log',
       'terminated_job_loss_noself_log', 'IDC_loss_log', 'IDC_econ_loss_log',
       'IDC_job_loss_log', 'grant_funds_log', 'grant_funds_econ_log',
       'combined_loss_log', 'combined_econ_loss_log', 'combined_job_loss_log',
       'combined_loss_noself_log', 'rep_name', 'pol_party'],
      dtype='ob

## Process Congressional Districts

In [10]:
congs = gpd.read_file('/home/colin/code/scimap-data/inputs/CongDist_shp/cb_2022_us_cd118_500k.shp') # Example, replace with your shapefile path
congs = congs.to_crs(epsg=4326) # Project to WGS84 (lat/lon)
congs['GEOID'] = congs['GEOID'].astype('int64')
# Merge the dataframes based on the FIPS code
# merged_df_cong_idc = pd.merge(df_idc_cong, congs[['GEOID', 'geometry']], on='GEOID', how='left')
# merged_df_cong_term =  pd.merge(df_term_cong, congs[['GEOID', 'geometry']], on='GEOID', how='left')
merged_df_cong_total =  pd.merge(df_total_cong, congs[['GEOID', 'geometry']], on='GEOID', how='left')

In [11]:
# features = []

# for index, row in tqdm(merged_df_cong_idc.iterrows(), desc="Processing into GeoJSON (IDC CONG)", total=len(merged_df_cong_idc)):
#     # Convert the Shapely geometry to GeoJSON format
#     geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
#     geometry = geometry['features'][0]['geometry']

#     # Create a feature dictionary
#     feature = {
#         "type": "Feature",
#         "geometry": geometry,
#         "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
#     }
#     features.append(feature)

# # Create the GeoJSON feature collection
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Save the GeoJSON to a file
# with open("/home/colin/code/scimap-data/outputs/merged_data_congs_idc.geojson", "w") as f:
#     json.dump(geojson, f)

# features = []

# for index, row in tqdm(merged_df_cong_term.iterrows(), desc="Processing into GeoJSON (TERM CONG)", total=len(merged_df_cong_idc)):
#     # Convert the Shapely geometry to GeoJSON format
#     geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
#     geometry = geometry['features'][0]['geometry']

#     # Create a feature dictionary
#     feature = {
#         "type": "Feature",
#         "geometry": geometry,
#         "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
#     }
#     features.append(feature)

# # Create the GeoJSON feature collection
# geojson = {
#     "type": "FeatureCollection",
#     "features": features
# }

# # Save the GeoJSON to a file
# with open("/home/colin/code/scimap-data/outputs/merged_data_congs_term.geojson", "w") as f:
#     json.dump(geojson, f)



features = []

for index, row in tqdm(merged_df_cong_total.iterrows(), desc="Processing into GeoJSON (TOTAL CONG)", total=len(merged_df_cong_total)):
    # Convert the Shapely geometry to GeoJSON format
    geometry = json.loads(gpd.GeoSeries([row['geometry']]).to_json())
    geometry = geometry['features'][0]['geometry']

    # Create a feature dictionary
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": row.drop('geometry').to_dict() # Include all columns except 'geometry'
    }
    features.append(feature)

# Create the GeoJSON feature collection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON to a file
with open("/home/colin/code/scimap-data/outputs/merged_data_congs_total.geojson", "w") as f:
    json.dump(geojson, f)

Processing into GeoJSON (TOTAL CONG):   0%|          | 0/436 [00:00<?, ?it/s]

# Turn into Tiles

In [12]:
VERSION = 21
os.chdir('/home/colin/code/scimap-data')

print("Processing States (TOTAL)")
!tippecanoe -z7 -e tiles_states_total_v{VERSION} --drop-densest-as-needed --no-tile-size-limit --no-tile-compression outputs/merged_data_states_total.geojson


print("Processing Districts (TOTAL)")
!tippecanoe -z7 -e tiles_districts_total_v{VERSION}  --drop-densest-as-needed --no-tile-size-limit --no-tile-compression outputs/merged_data_congs_total.geojson

print("Processing Counties (TOTAL)")
!tippecanoe -z7 -e tiles_counties_total_v{VERSION} --drop-densest-as-needed --no-tile-size-limit --no-tile-compression outputs/merged_data_counties_total.geojson

Processing States (TOTAL)
For layer 0, using name "merged_data_states_total"
51 features, 3219117 bytes of geometry and attributes, 28772 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
  99.9%  7/36/48   
Processing Districts (TOTAL)
For layer 0, using name "merged_data_congs_total"
436 features, 1962268 bytes of geometry and attributes, 292466 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
  99.9%  7/29/51   
Processing Counties (TOTAL)
For layer 0, using name "merged_data_counties_total"
3144 features, 6887617 bytes of geometry and attributes, 1305055 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
  99.9%  7/29/52  


In [22]:
print("Processing Counties (TOTAL)")
!tippecanoe -z7 -e tiles_counties_total_v{VERSION} --no-tile-compression outputs/merged_data_counties_total.geojson --force

Processing Counties (TOTAL)
For layer 0, using name "merged_data_counties_total"
    3144 features, 6839079 bytes of geometry and attributes, 1285763 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
tile 0/0/0 size is 1176650 (probably really 1176650) with detail 12, >500000    
tile 0/0/0 size is 1112979 (probably really 1112979) with detail 11, >500000    
tile 0/0/0 size is 586201 (probably really 586201) with detail 10, >500000    
tile 1/0/0 size is 1203610 (probably really 1203610) with detail 12, >500000    
tile 1/0/0 size is 1177192 (probably really 1177192) with detail 11, >500000    
tile 1/0/0 size is 1112891 (probably really 1112891) with detail 10, >500000    
tile 1/0/0 size is 585617 (probably really 585617) with detail 9, >500000    
tile 2/1/1 size is 685493 (probably really 685493) with detail 12, >500000    
tile 2/0/1 size is 710320 (probably really 710320) with detail 12, >500000    
tile 2/1/1 size is 662528 (probably really 662528) with detail 11, >50

## Upload Tiles

In [14]:
!rclone copy tiles_states_total_v{VERSION}/ r2:scimap-data/tiles_states_total_v{VERSION}/ --transfers 32
!rclone copy tiles_counties_total_v{VERSION}/ r2:scimap-data/tiles_counties_total_v{VERSION}/ --transfers 32
!rclone copy tiles_districts_total_v{VERSION}/ r2:scimap-data/tiles_districts_total_v{VERSION}/ --transfers 32


In [28]:
## TOP LEVEL STATS

## Top level stats for Cancellations + Future Losses

In [15]:
df_total_state.head()

,state,state_code,terminated_loss,terminated_loss_noself,terminated_econ_loss,terminated_job_loss,terminated_econ_loss_noself,terminated_job_loss_noself,IDC_loss,IDC_econ_loss,...,terminated_job_loss_noself_log,IDC_loss_log,IDC_econ_loss_log,IDC_job_loss_log,grant_funds_log,grant_funds_econ_log,combined_loss_log,combined_econ_loss_log,combined_job_loss_log,combined_loss_noself_log
0,Alabama,AL,1.956998e+07,1.956291e+07,5.009915e+07,216.385678,5.008106e+07,216.307564,5.357652e+07,1.371559e+08,...,5.381314,17.796621,18.736629,6.385863,19.768473,20.708480,18.096642,19.036552,6.685349,18.096544
1,Alaska,AK,1.428496e+06,1.427279e+06,3.656949e+06,15.794906,3.653835e+06,15.781458,1.339256e+06,3.428495e+06,...,2.820275,14.107625,15.047632,2.760527,16.520911,17.460919,14.833037,15.772607,3.452339,14.832600
2,Arizona,AZ,1.710930e+07,1.662762e+07,4.379980e+07,189.177866,4.256670e+07,183.851934,5.233332e+07,1.339733e+08,...,5.219555,17.773144,18.713151,6.362425,19.663902,20.603909,18.051864,18.985727,6.634590,18.045720
3,Arkansas,AR,4.039129e+06,4.035052e+06,1.034017e+07,44.660737,1.032973e+07,44.615658,1.810754e+07,4.635530e+07,...,3.820251,16.711839,17.651846,5.304376,18.559582,19.499589,16.904043,17.843865,5.495526,16.903857
4,California,CA,3.608646e+08,3.436208e+08,9.238134e+08,3990.087872,8.796693e+08,3799.422634,8.873075e+08,2.271507e+09,...,8.242868,20.603702,21.543709,9.191359,22.354541,23.294548,20.935284,21.861283,9.508905,20.921276


In [16]:
state_losses = {}
for f in df_total_state.to_dict("records"):
    state_losses[f["state"]] = {
        "state_code": f["state_code"],
        "idc_loss": f["IDC_econ_loss"],
        "term_loss": f["terminated_econ_loss"],
        "idc_job_loss": f["IDC_job_loss"],
        "term_job_loss": f["terminated_job_loss"],
        }

In [17]:
state_losses

{'Alabama': {'state_code': 'AL',
  'idc_loss': 137155882.470028,
  'term_loss': 49996588.2760707,
  'idc_job_loss': 592.39668385991,
  'term_job_loss': 215.942711064731},
 'Alaska': {'state_code': 'AK',
  'idc_loss': 3428494.51879316,
  'term_loss': 2923061.33923361,
  'idc_job_loss': 14.8081784535109,
  'term_job_loss': 12.6251372737111},
 'Arizona': {'state_code': 'AZ',
  'idc_loss': 133973289.33053,
  'term_loss': 43135728.607902,
  'idc_job_loss': 578.650590087181,
  'term_job_loss': 186.309636327747},
 'Arkansas': {'state_code': 'AR',
  'idc_loss': 46355302.6789298,
  'term_loss': 10266784.0486183,
  'idc_job_loss': 200.215456251548,
  'term_job_loss': 44.343769401479},
 'California': {'state_code': 'CA',
  'idc_loss': 2271507094.07446,
  'term_loss': 704792197.418731,
  'idc_job_loss': 9810.9774488748,
  'term_job_loss': 3044.10246970218},
 'Colorado': {'state_code': 'CO',
  'idc_loss': 222803650.787174,
  'term_loss': 35966203.4953647,
  'idc_job_loss': 962.322151272262,
  'term

In [1]:
## Congressional Patch (temp)

In [3]:
import pandas as pd
cong_patch = pd.read_csv("/home/colin/Downloads/USA_119th_Congressional_Districts_V2.csv")

In [6]:
cong_patch_json = {}
for row in cong_patch.to_dict("records"):
    cong_patch_json[row["GeoID"]] = {
        "rep_name": row["Full_Name"],
        "rep_party": row["Party"],

        }

In [7]:
cong_patch_json

{101: {'rep_name': 'Jerry Carl', 'rep_party': 'Republican'},
 102: {'rep_name': 'Shomari Figures', 'rep_party': 'Democrat'},
 103: {'rep_name': 'Mike Rogers', 'rep_party': 'Republican'},
 104: {'rep_name': 'Robert B. Aderholt', 'rep_party': 'Republican'},
 105: {'rep_name': 'Dale W. Strong', 'rep_party': 'Republican'},
 106: {'rep_name': 'Gary J. Palmer', 'rep_party': 'Republican'},
 107: {'rep_name': 'Terri A. Sewell', 'rep_party': 'Democrat'},
 200: {'rep_name': 'Nick Begich III', 'rep_party': 'Republican'},
 401: {'rep_name': 'David Schweikert', 'rep_party': 'Republican'},
 402: {'rep_name': 'Elijah Crane', 'rep_party': 'Republican'},
 403: {'rep_name': 'Yassamin Ansari', 'rep_party': 'Democrat'},
 404: {'rep_name': 'Greg Stanton', 'rep_party': 'Democrat'},
 405: {'rep_name': 'Andy Biggs', 'rep_party': 'Republican'},
 406: {'rep_name': 'Juan Ciscomani', 'rep_party': 'Republican'},
 407: {'rep_name': 'Raul M. Grijalva*', 'rep_party': 'Democrat'},
 408: {'rep_name': 'Abraham Hamadeh',